# COCO 스러운 Dataset 형식 만들기

In [13]:
import datetime
import json
import os
import re
import fnmatch
from PIL import Image
import numpy as np
import sys
sys.path.insert(0,'./../../')
from pycococreatortools import pycococreatortools
import csv

In [ ]:
ROOT_DIR = 'train'
IMAGE_DIR = os.path.join(ROOT_DIR, "KNU_train2018")
ANNOTATION_DIR = os.path.join(ROOT_DIR, "annotations")

    INFO와 LICENSES는 안걸들려도됨

In [15]:
INFO = {
    "description": "Example Dataset",
    "url": "https://github.com/waspinator/pycococreator",
    "version": "0.1.0",
    "year": 2018,
    "contributor": "waspinator",
    "date_created": datetime.datetime.utcnow().isoformat(' ')
}

In [16]:
LICENSES = [
    {
        "id": 1,
        "name": "Attribution-NonCommercial-ShareAlike License",
        "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
    }
]

    지금 카테고리가 세개임 (shape 예제 그대로 가져와서)
    이 예제는 카테고리 id가 1부터 시작하는데
    가령 우리 예제는 0부터 시작한다던가 하면 coco에서 계산할때 0은 무시해버림
    1,2,3 제외한 caterory id들은 전부다 무시해버림

In [17]:
CATEGORIES = [
    {
        'id': 1,
        'name': 'square',
        'supercategory': 'shape',
    },
    {
        'id': 2,
        'name': 'circle',
        'supercategory': 'shape',
    },
    {
        'id': 3,
        'name': 'triangle',
        'supercategory': 'shape',
    },
]

In [19]:
def create_annotation_info(annotation_id, image_id, category_info, bounding_box, 
                           image_size, tolerance=2, binary_mask = None):

    
    segmentation = [[1]]

    area = 10

    if category_info["is_crowd"]:
        is_crowd = 1
    else :
        is_crowd = 0


    annotation_info = {
        "id": annotation_id,
        "image_id": image_id,
        "category_id": category_info["id"],
        "iscrowd": is_crowd, #iscrowd 옵션에 따라서 어떻게 변하는지 잘 모르겠음 
        "area": bounding_box[2] *  bounding_box[2],
        "bbox": bounding_box,
        #"segmentation": segmentation, #segmentation 안할꺼면 굳이 안넣어도됨 
        "width": image_size[0],
        "height": image_size[1]
    } 
    
    return annotation_info

In [20]:
image_id = 1
segmentation_id = 1

In [21]:
def filter_for_jpeg(root, files):
    file_types = ['*.jpeg', '*.jpg']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    
    return files

In [22]:
def filter_for_annotations(root, files, image_filename):
    file_types = ['*.txt']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    basename_no_extension = os.path.splitext(os.path.basename(image_filename))[0]
    file_name_prefix = basename_no_extension + '.*'
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    files = [f for f in files if re.match(file_name_prefix, os.path.splitext(os.path.basename(f))[0])]

    return files

In [23]:
coco_output = {
        "info": INFO,
        "licenses": LICENSES,
        "categories": CATEGORIES,
        "images": [],
        "annotations": []
    }

In [25]:
IMAGE_DIR = './images'
ANNOTATION_DIR = "./annotations"
image_id = 1
object_id = 1
for root, _, files in os.walk(IMAGE_DIR):
        image_files = filter_for_jpeg(root, files)

        # go through each image
        for image_filename in image_files:
            image = Image.open(image_filename)
            image_info = pycococreatortools.create_image_info(
                image_id, os.path.basename(image_filename), image.size)
            coco_output["images"].append(image_info)

            # filter for associated png annotations
            for root, _, files in os.walk(ANNOTATION_DIR):
                annotation_files = filter_for_annotations(root, files, image_filename)

                # go through each associated annotation
                for annotation_filename in annotation_files:
                    print(annotation_filename)
                    
                    with open(annotation_filename, 'r') as raw:
                        lines = raw.readlines()
                    cooked = csv.reader(lines)
                    for record in cooked:
                        print(record)
                        my_list = list(np.float_(record[0].split(' ')))
                        print(my_list)
                        category_info = {'id': int(my_list[0]), 'is_crowd': False}
                        
                        height =image.size[1]
                        width = image.size[0]
                        
                        object_width = my_list[3] * width
                        object_height = my_list[4] * height
                        
                        center_x =  my_list[1] * width
                        center_y = my_list[2] * height
                        
                        topleft_x = center_x - object_width / 2
                        topleft_y = center_y - object_height / 2
                        
                        print("center_x:",center_x)
                        print("center_y:",center_y)
                        print("width:",width)
                        print("height:",height)
                        print(topleft_x,topleft_y,object_width,object_height)
                        
                        annotation_info = create_annotation_info(
                        annotation_id = object_id, image_id = image_id, category_info = category_info, 
                            bounding_box = [topleft_x,topleft_y,object_width,object_height],
                            image_size = image.size, tolerance=2, binary_mask = None)

                        if annotation_info is not None:
                            coco_output["annotations"].append(annotation_info)
              
                        object_id = object_id + 1
            image_id = image_id + 1
        
        

./annotations\20180312_170057_0000.txt
['1 0.947656 0.066667 0.062500 0.130556']
[1.0, 0.947656, 0.066667, 0.0625, 0.130556]
center_x: 3638.99904
center_y: 144.00072
width: 3840
height: 2160
3518.99904 3.000239999999991 240.0 282.00096
./annotations\20180312_170057_0001.txt
['2 0.950781 0.061806 0.050000 0.120833']
[2.0, 0.950781, 0.061806, 0.05, 0.120833]
center_x: 3650.99904
center_y: 133.50096
width: 3840
height: 2160
3554.99904 3.0013199999999927 192.0 260.99928
./annotations\20180312_170057_0002.txt
['0 0.950781 0.058333 0.053125 0.113889']
[0.0, 0.950781, 0.058333, 0.053125, 0.113889]
center_x: 3650.99904
center_y: 125.99928000000001
width: 3840
height: 2160
3548.99904 2.9991600000000034 204.0 246.00024000000002
./annotations\20180312_170057_0003.txt
['0 0.946484 0.065972 0.053906 0.129167']
[0.0, 0.946484, 0.065972, 0.053906, 0.129167]
center_x: 3634.49856
center_y: 142.49952000000002
width: 3840
height: 2160
3530.99904 2.9991600000000176 206.99904 279.00072
./annotations\201803

# 파일 쓰기
    만든 coco_output를 가지고 최종 파일 만들어냄

In [26]:
with open('{}/instances_KNU_train2018.json'.format(ROOT_DIR), 'w') as output_json_file:
    json.dump(coco_output, output_json_file)

In [27]:
#print(coco_output)